In [1]:
import sys
sys.path.append('../src')
sys.path.append('../scripts')

In [2]:
from jobDescriptionScraper import LinkedInDescriptionScraper
from jobListingScraper import LinkedInJobListingScraper

In [3]:
%%time
# Job Listing Scraping
job_listing_scraper = LinkedInJobListingScraper()
jobs = job_listing_scraper.scrape_linkedin_jobs("Fullstack Developer", "India", 50, "../linkedincookies.pkl")

Scraping page starting at 0
Scraping page starting at 25
CPU times: total: 312 ms
Wall time: 33.2 s


In [4]:
%%time
# Job Description Scraping
job_description_scraper = LinkedInDescriptionScraper()
jobs['Job Descriptions'] = jobs['Job ID'].apply(lambda x : job_description_scraper.getDescription(job_id=x))

CPU times: total: 141 ms
Wall time: 1min 9s


In [5]:
old_resume = """
1
John Anderson

Email: john.anderson@gmail.com | Phone: (555) 555-1234 | LinkedIn: linkedin.com/in/johnanderson |
GitHub: github.com/johndevml | Location: San Francisco, CA

Professional Summary

MLOps Engineer with 2+ years of experience in deploying and managing machine learning models,
computer vision systems, and generative AI models in production. Proficient in automating end-to-end
machine learning pipelines and integrating advanced AI solutions using cutting-edge tools like Docker,
Kubernetes, TensorFlow, PyTorch, and MLflow. Strong expertise in scaling AI systems using cloud
platforms (AWS, GCP), and optimizing deep learning models for real-time inference. Adept at
integrating MLOps best practices, such as CI/CD pipelines and monitoring frameworks, to improve
model robustness and scalability in production environments. Experienced in working with GANs and
other generative models for various applications in image synthesis and data augmentation.

Skills

Mlops tools:

Docker, Kubernetes, Jenkins, MLflow, Airflow, TensorBoard, Argo Workflows

Machine learning:

Supervised/Unsupervised Learning, Model Deployment, Hyperparameter Tuning

Computer vision:

OpenCV, TensorFlow, PyTorch, YOLO, Faster R-CNN, Mask R-CNN

Generative ai:

GANs, Variational Autoencoders (VAEs), DALL-E, Stable Diffusion

Cloud platforms:

AWS (EC2, S3, Lambda, SageMaker), GCP (AI Platform, Compute Engine)

Programming languages:

Python, Bash, SQL, C++

Version control ci cd:

Git, GitLab CI, Jenkins, CircleCI, GitHub Actions

Containerization orchestration:

Docker, Kubernetes

Data engineering:

Apache Kafka, Apache Spark, Hadoop

Monitoring logging:

Prometheus, Grafana, ELK Stack

Professional Experience

MLOps Engineer

XYZ Corp – San Francisco, CA

June 2022 – Present

• Architected and deployed real-time object detection models for autonomous vehicle systems, using
TensorFlow and YOLOv5, enabling the detection of pedestrians and vehicles with 95% accuracy.

• Built and optimized deep learning pipelines for computer vision tasks, including image classification
and segmentation, using Kubernetes for large-scale deployment on AWS.
• Automated the deployment and retraining of generative models, such as GANs for synthetic image
generation, improving the model development lifecycle by 30%.

• Developed a CI/CD framework using Jenkins and MLflow to automate the training and deployment of
computer vision models, reducing deployment time from weeks to days.

• Implemented GPU acceleration in model inference pipelines on AWS SageMaker, leading to a 50%
reduction in processing time for complex computer vision models.

• Integrated TensorBoard and Prometheus for real-time model performance monitoring, ensuring model
accuracy and reducing drift in production by 20%.

Junior MLOps Engineer

DataTech Solutions – Los Angeles, CA

August 2021 – June 2022

• Designed and maintained automated pipelines for the deployment of generative AI models (e.g.,
DCGANs) to enhance datasets through synthetic data generation.

• Containerized machine learning models for natural language processing (NLP) tasks such as text
classification and summarization using Docker and Kubernetes.

• Led the deployment of computer vision models for image recognition in retail environments, which
improved product identification accuracy by 15%.

• Collaborated with data science teams to implement transfer learning for custom object detection
tasks, shortening the model development cycle by 40%.

• Created TensorFlow Serving pipelines for seamless model deployment across development and
production environments, ensuring consistent API responses.

• Implemented logging and error-handling mechanisms in real-time NLP systems using ELK Stack,
reducing system downtime by 25%.

Projects

Automated ML Pipeline for Predictive Maintenance in Autonomous Vehicles

XYZ Corp

Developed a real-time object detection pipeline for predicting maintenance issues in autonomous
vehicles using YOLOv5 and TensorFlow. Integrated Airflow for orchestrating tasks such as model
training, validation, and deployment on AWS Lambda, enabling real-time updates to the model with
minimal downtime. Monitored model drift and performance using Prometheus and automated retraining
when accuracy dropped, ensuring model precision stayed above 90%.

Real-time Fraud Detection System Using Generative AI

DataTech Solutions

Built a fraud detection system that utilized variational autoencoders (VAEs) to detect anomalies in
real-time transaction data streams. Deployed the model as a microservice on Kubernetes, leveraging
horizontal scaling to support high-traffic environments. Used MLflow for model versioning and Jenkins
to automate CI/CD workflows for model updates. The system achieved a fraud detection rate of 97%,
significantly improving upon existing rule-based systems.

Generative Image Synthesis Pipeline

XYZ Corp
"""

In [6]:
# Old Resume Scoring
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

jobs['Old Resume Score'] = jobs['Job Descriptions'].apply(lambda x : resume_relevence_scorer.calculate_relevance_score(old_resume, x))

c:\Users\mnsnn\anaconda3\envs\regen\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\mnsnn\anaconda3\envs\regen\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
generated_json_data = {
  "name": "John Anderson",
  "contact": {
    "email": "john.anderson@gmail.com",
    "linkedin": "linkedin.com/in/johnanderson",
    "github": "github.com/johndevml",
    "phone": "(555) 555-1234",
    "location": "India"
  },
  "summary": "Senior MLOps Engineer with 7+ years of experience in deploying, managing, and optimizing machine learning pipelines and systems at scale. Expertise in building and scaling AI solutions using Docker, Kubernetes, and cloud platforms like AWS and GCP. Skilled in automating end-to-end MLOps workflows, implementing CI/CD pipelines, and leveraging advanced AI tools including TensorFlow and PyTorch. Proven track record in ensuring model performance and reliability through real-time monitoring and optimization, with a focus on problem-solving and continuous improvement.",
  "skills": {
    "mlops_tools": [
      "Docker",
      "Kubernetes",
      "Jenkins",
      "MLflow",
      "Airflow",
      "Argo Workflows",
      "Terraform",
      "Ansible"
    ],
    "machine_learning": [
      "Supervised/Unsupervised Learning",
      "Model Deployment",
      "Model Performance Monitoring"
    ],
    "cloud_platforms": [
      "AWS (EC2, S3, Lambda, SageMaker, EMR, Kinesis, SQS, SNS)",
      "GCP (AI Platform, Compute Engine)"
    ],
    "programming_languages": [
      "Python",
      "SQL"
    ],
    "version_control_ci_cd": [
      "Git",
      "Jenkins",
      "GitHub Actions",
      "TravisCI",
      "Bamboo"
    ],
    "data_engineering": [
      "Apache Spark",
      "Airflow"
    ]
  },
  "experience": [
    {
      "title": "MLOps Engineer",
      "company": "XYZ Corp",
      "location": "India",
      "duration": "June 2022 – Present",
      "responsibilities": [
        "Architected and deployed real-time object detection models for autonomous vehicle systems, using TensorFlow and YOLOv5, with a focus on scalability and performance.",
        "Built and optimized deep learning pipelines on AWS using Kubernetes, enhancing model deployment efficiency and scalability.",
        "Automated deployment and retraining of generative models, including GANs, improving the model development lifecycle.",
        "Developed CI/CD frameworks with Jenkins and MLflow, reducing deployment times and ensuring reliable and maintainable production systems.",
        "Implemented GPU acceleration in model inference pipelines on AWS SageMaker, optimizing processing times and model performance.",
        "Integrated TensorBoard and Prometheus for real-time performance monitoring, ensuring model accuracy and operational reliability."
      ]
    },
    {
      "title": "Junior MLOps Engineer",
      "company": "DataTech Solutions",
      "location": "India",
      "duration": "August 2021 – June 2022",
      "responsibilities": [
        "Designed and managed automated pipelines for deploying generative AI models, enhancing datasets through synthetic data generation.",
        "Containerized NLP models using Docker and Kubernetes for scalable deployment.",
        "Led deployment of computer vision models, improving accuracy and operational efficiency.",
        "Implemented transfer learning for object detection tasks, accelerating the model development process.",
        "Created TensorFlow Serving pipelines for consistent and scalable model deployment.",
        "Implemented logging and error-handling mechanisms using ELK Stack, reducing system downtime."
      ]
    }
  ],
  "projects": [
    {
      "name": "Automated ML Pipeline for Predictive Maintenance in Autonomous Vehicles",
      "company": "XYZ Corp",
      "description": "Developed a real-time object detection pipeline using YOLOv5 and TensorFlow for predicting maintenance issues. Integrated Airflow for orchestrating model training and deployment on AWS Lambda, ensuring high availability and minimal downtime."
    },
    {
      "name": "Real-time Fraud Detection System Using Generative AI",
      "company": "DataTech Solutions",
      "description": "Built a fraud detection system utilizing VAEs for real-time anomaly detection. Deployed on Kubernetes with CI/CD automation using Jenkins, achieving high accuracy and scalability."
    },
    {
      "name": "Generative Image Synthesis Pipeline",
      "company": "XYZ Corp",
      "description": "Developed a GAN-based image synthesis pipeline to augment training datasets, deployed on GCP AI Platform with TensorFlow and Kubernetes, improving model performance and training efficiency."
    }
  ],
  "open_source_contributions": [
    {
      "project": "Kubeflow",
      "contribution": "Developed custom operators for scaling computer vision pipelines on Kubernetes."
    },
    {
      "project": "MLflow",
      "contribution": "Created extensions for tracking and visualizing GAN training metrics."
    },
    {
      "project": "TensorFlow Hub",
      "contribution": "Developed pre-trained models for image classification and segmentation."
    }
  ],
  "education": {
    "degree": "Bachelor of Science in Computer Science",
    "institution": "University of California, Berkeley",
    "graduation_year": "2020",
    "relevant_courses": [
      "Machine Learning",
      "Deep Learning",
      "Cloud Computing",
      "Distributed Systems"
    ]
  },
  "certifications": [
    {
      "name": "AWS Certified Solutions Architect – Associate",
      "issued": "May 2023"
    },
    {
      "name": "Certified Kubernetes Administrator (CKA)",
      "issued": "September 2022"
    },
    {
      "name": "TensorFlow Developer Certification",
      "issued": "March 2022"
    },
    {
      "name": "Google Cloud Professional Data Engineer",
      "issued": "February 2024"
    }
  ],
  "publications_talks": [
    {
      "title": "Building and Deploying Computer Vision Models at Scale with TensorFlow and Kubernetes",
      "event": "O'Reilly AI Conference",
      "year": "2023"
    },
    {
      "title": "Generative AI in Real-World Applications: Challenges and Solutions",
      "platform": "Medium",
      "views": "20,000+"
    },
    {
      "title": "Optimizing Deep Learning Workflows with Airflow and MLflow",
      "event": "PyData Conference",
      "year": "2023"
    }
  ],
  "volunteer_experience": [
    {
      "organization": "AI for Good Hackathon",
      "role": "Mentor",
      "description": "Mentored teams in designing and deploying computer vision models for healthcare applications."
    },
    {
      "organization": "Women Who Code",
      "role": "Workshop Leader",
      "description": "Conducted workshops on Generative AI with a focus on using GANs and VAEs for image synthesis."
    }
  ],
  "technical_proficiencies": {
    "operating_systems": ["Linux (Ubuntu, CentOS)", "Windows"],
    "databases": ["MySQL", "PostgreSQL"],
    "version_control": ["Git"]
  },
  "references": "Available upon request."
}


In [8]:
# PDF Generation from JSON for each JOB Description
from resumeGenerator import generate_resume_from_json

jobs['Generated Json Data'] = None
jobs['Generated Resume Path'] = jobs[['Job Descriptions', 'Job ID']].apply(lambda x : generate_resume_from_json(generated_json_data, x[1]), axis=1)

C:\Users\mnsnn\AppData\Local\Temp\ipykernel_26348\1001097404.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  jobs['Generated Resume Path'] = jobs[['Job Descriptions', 'Job ID']].apply(lambda x : generate_resume_from_json(generated_json_data, x[1]), axis=1)


In [9]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score,Generated Json Data,Generated Resume Path
0,"(IND) SENIOR, SOFTWARE ENGINEER - Machine Lear...",https://www.linkedin.com/jobs/view/3994669548/...,Walmart Global Tech India,"Bengaluru, Karnataka, India (On-site)",3994669548,Position Summary...\n\nDemonstrates up-to-date...,59.36,None,../generatedResumes\3994669548.pdf
1,Java FullStack developer,https://www.linkedin.com/jobs/view/4005724254/...,Wipro,"Chennai, Tamil Nadu, India (On-site)",4005724254,Role: Java Full Stack Developer (Angular or Re...,65.72,None,../generatedResumes\4005724254.pdf
2,Full-Stack Developer [1 month NP MAX],https://www.linkedin.com/jobs/view/3918301973/...,Luxoft,"Bengaluru, Karnataka, India (On-site)",3918301973,Project Description:\nLuxoft has been asked to...,68.95,None,../generatedResumes\3918301973.pdf
3,Full Stack Developer - Team Lead Module Lead,https://www.linkedin.com/jobs/view/4021499977/...,Oracle,"Trivandrum, Kerala, India (On-site)",4021499977,Job Description\n\nAn experienced consulting p...,63.40,None,../generatedResumes\4021499977.pdf
4,Full Stack Developer,https://www.linkedin.com/jobs/view/4020481927/...,IBM,"Kochi, Kerala, India (On-site)",4020481927,"Introduction\n\nAt IBM, work is more than a jo...",69.86,None,../generatedResumes\4020481927.pdf
5,Fullstack Engineer,https://www.linkedin.com/jobs/view/4005645994/...,IBM,"Kochi, Kerala, India (On-site)",4005645994,"Introduction\n\nAt IBM, work is more than a jo...",67.27,None,../generatedResumes\4005645994.pdf
6,"Senior Software Engineer, Python Full-Stack, I...",https://www.linkedin.com/jobs/view/3940378528/...,BCG X,"Bengaluru, Karnataka, India (Hybrid)",3940378528,Who We Are\n\nBoston Consulting Group partners...,66.51,None,../generatedResumes\3940378528.pdf
7,Dotnet Fullstack Developer,https://www.linkedin.com/jobs/view/4001605143/...,Tata Consultancy Services,"Bengaluru, Karnataka, India (On-site)",4001605143,Greetings from TCS !!!\n\nWalkin Interview for...,68.71,None,../generatedResumes\4001605143.pdf
8,Full Stack Developer ( React + .NET Core + Imm...,https://www.linkedin.com/jobs/view/3963368766/...,UST,"Trivandrum, Kerala, India (Hybrid)",3963368766,Basic Qualifications:\n• Preferably 6 plus yea...,68.12,None,../generatedResumes\3963368766.pdf
9,Java Fullstack Developer,https://www.linkedin.com/jobs/view/4001675004/...,Tata Consultancy Services,Greater Bengaluru Area (On-site),4001675004,Greetings from TCS!\n\nJob skill: Java Fullsta...,70.48,None,../generatedResumes\4001675004.pdf
